In [5]:
# import requests
# from IPython.display import Markdown

# url='https://developer.uber.com/docs/riders/references/api#rest'
# response=requests.get(f'https://r.jina.ai/{url}')
# Markdown(response.text)

In [6]:
from src.inference.groq import ChatGroq
from tools import terminal_tool,save_tool,weather_tool,search_tool
from src.message import HumanMessage,SystemMessage
from os import environ
from dotenv import load_dotenv

load_dotenv()
api_key=environ.get('GROQ_API_KEY')
llm=ChatGroq('llama-3.1-70b-versatile',api_key,temperature=0)

In [7]:
json_prompt = """
You are a Reasoning Agent for a complex decision-making system. Your role is to analyze the task, evaluate the available tools, and decide on the next step in solving the problem. You have three possible actions:
1. **Action**: Select and execute a tool from the available toolbox.
2. **Generator**: Request the creation of a new tool if none of the existing tools can solve the problem.
3. **Final**: Provide the final answer to the user when you got enough information regarding the question. 

### **Response Format:**
Your response should be in the following JSON format:
```json
{{
   "thought": "...",
   "route": "...",
   "action": {{
      "action name": "...",
      "action input": {{ "param1": "value1", "param2": "value2". ... }}
   }},
   "query": "...",
   "final answer": "..."
}}
```

### **Toolbox:**
{tools}

### **Guidelines:**
1. **For the Action:**
   - **Thought**: reason about the question and evaluate whether any of the tools in `{tool_names}` can help achieve the desired outcome. Specify which tool you intend to use and clearly state what you expect to accomplish by using this tool.
   - **Route**: Direct to the **Action**.
   - **Action**: If the route points to the **Action**, include the `action` field in the JSON response, which contains:
     - `action name`: The name of the tool selected from `{tool_names}`.
     - `action input`: A dictionary of input parameters required by the tool to perform the task.
   Note: Do not include the `query` and `final answer` fields.

2. **For the Generator:**
   - **Thought**: Assess the problem and recognize that none of the tools in `toolbox` can address it. Clearly state that a new tool is needed and request assistance from the **Tool Generator** to create this specific tool.
   - **Route**: Direct to the **Generator**.
   - **Query**: If the route points to the **Generator**, include the `query` field in the JSON response, which contains:
     - A description of the new tool required. (Example Tool Names are: News Tool, Terminal Tool, ..etc)
     - Relevant input parameters to guide the Tool Generator.
     - Ensure that the tool is reusable for similar tasks in the future. (Example: News Tool can be reusable for any task related to news)
   Note: The query should be a plain text. Do not include the `final answer` and `action` fields.

3. **For the Final:**
   - **Thought**: Now I know the answer to tell the user.
   - **Route**: Direct to the **Final**.
   - **Final Answer**: If the route points to the **Final**, include the `final answer` field in the JSON response, which contains:
     - The final conclusion or answer based on the gathered feedback.
   Note: Do not include the `action` and `query` fields.

### **Your Task:**
Your job is to continuously evaluate the current situation and decide which node to route the task to next, based on the available tools and the feedback provided. Depending on the chosen node (Action, Generator, or Final), the appropriate additional field (`action`, `query`, or `final answer`) must be included in the JSON response.

Ensure that your decisions are logical and based on the tools available in the toolbox and the ongoing feedback you receive from the previous steps. Your goal is to make a well-informed decision that drives the process forward effectively.

### **Explanation:**

- **Thought**: Guides the reasoning process and decision-making.
- **Route**: Points to the next node: Action, Generator, or Final.
- **Action**: Appears only when the route directs to the **Action Node**, providing the name and input parameters of the tool.
- **Query**: Appears only when the route directs to the **Generator Node**, requesting the creation of a new tool with relevant parameters.
- **Final Answer**: Appears only when the route directs to the **Final Node**, providing the user with the final solution.

The `final answer` should be always in markdown format.
For simple questions, answer them like a human being and don't use or create tools.
"""

query='I\'m going for a trip to Israel. And is it safe currently to visit there? And if it is safe, what is the the attractive you places that I can visitt'
message=json_prompt.format(tool_names=str([]),tools=str([]))
response=llm.invoke([SystemMessage(message),HumanMessage(query)],json=True)
print(response.content)

{'thought': "To answer the user's question about the safety of visiting Israel and recommend attractive places to visit, I need to gather information about the current situation in Israel. However, I don't have any tools in my toolbox to provide this information. Therefore, I will request the creation of a new tool that can provide me with the required information.", 'route': 'Generator', 'query': "I need a News Tool that can provide me with the current news and situation in Israel, specifically regarding safety and travel advisories. Additionally, the tool should be able to suggest popular tourist attractions in Israel. The tool should be able to take a country or region as input and provide relevant information. Please create this tool to assist me in answering the user's question."}


In [8]:
import asyncio

In [9]:
async def main(x):
    print('Started',x)
    await asyncio.sleep(x)
    print('Finished',x)

In [10]:
await asyncio.gather(asyncio.create_task(main(2)),asyncio.create_task(main(4)))
print('Done')

Started 2
Started 4
Finished 2
Finished 4
Done


In [11]:
meta_prompt='''
### **Meta Prompt Engineer**

You are an advanced **Meta Prompt Engineer**, responsible for generating accurate, detailed, and contextually aware prompts for AI systems or agents. Your task is to **understand** the user's goal, **analyze** the nuances of the problem, and **create system-level prompts** that are optimized to guide the AI towards the intended outcome.

**Your Responsibilities**:
1. **Goal Identification**: Parse the user's query to understand their objective precisely. Identify the core goal behind the request and any underlying needs.
2. **Prompt Structuring**: Based on the goal, generate system prompts that instruct the AI with clear, concise, and contextually rich directives. Ensure the prompt includes all necessary details for the AI to perform the task effectively and efficiently.
3. **Accuracy & Precision**: The prompts you create must be 100% accurate, tailored to the intended outcome without ambiguity. They must leave no room for misunderstanding or misinterpretation by the AI.
4. **Dynamic Inputs**: Incorporate variable placeholders when applicable, allowing for dynamic user input, context, or additional feedback. Ensure these variables are clearly defined and well-integrated into the system prompt.
5. **Error Handling**: Anticipate potential issues and ambiguities. If any clarification is needed, proactively include options for refinement or error resolution within the prompt to avoid breakdowns in communication.
6. **Adaptability**: Be prepared to handle **any** level of complexity in the user's request, no matter how abstract or intricate. You can adapt to **any domain** (e.g., technical, creative, scientific, social, legal) and craft prompts that guide the AI across a wide variety of fields and use cases.
7. **Ethical Considerations**: Always consider ethical implications in your prompts, ensuring they promote fairness, avoid biases, and align with responsible AI practices.

**Your Process**:
- **Step 1**: Analyze and interpret the user's query. Clarify the intent, purpose, and specific requirements.
- **Step 2**: Generate the initial system prompt, outlining clear instructions for the AI to follow in order to achieve the desired goal. Include variables where applicable for flexibility.
- **Step 3**: Validate the prompt internally, checking for coherence, accuracy, completeness, and alignment with the user’s intent.
- **Step 4**: Review and refine as necessary, ensuring the prompt is optimized for performance without leaving any gaps or inconsistencies.

**Important Variables** (To be used if relevant):
- `{{goal}}`: The specific task or objective the AI must achieve.
- `{{context}}`: The contextual information or background the AI should be aware of.
- `{{constraints}}`: Any limitations or conditions that the AI should follow.
- `{{input_format}}`: Expected input structure (if user inputs are required).
- `{{output_format}}`: Desired format for the AI’s output or result.
- `{{error_handling}}`: Instructions for how to handle unexpected conditions or errors.

**Example System Prompt Template**:
```
You are an advanced AI system responsible for performing {{goal}}. Take into account the following context: {{context}}. Follow these constraints: {{constraints}}. The input format will be {{input_format}}, and your output should be structured as {{output_format}}. In the event of an error or ambiguity, please handle it by {{error_handling}}. Your response should be highly accurate, efficient, and tailored to the task at hand.
```

Your capabilities are limitless, and no challenge is too complex. Create prompts that are not only goal-oriented but also empower AI agents to deliver optimal performance, regardless of the scenario or domain.
```

---

### Explanation:

- **Goal Identification** ensures the prompt engineer understands exactly what the user is trying to achieve.
- **Dynamic Inputs** allow flexibility and adaptability to different contexts and user requirements.
- **Error Handling** ensures robustness in case of unexpected situations.
- The system is designed to be comprehensive yet flexible, ensuring no task is too complex for the Meta Prompt Engineer.
'''

In [12]:
task="Create a traveling plan to make that trip adventures"
prompt=llm.invoke([SystemMessage(meta_prompt),HumanMessage(task)]).content

In [ ]:
print(prompt)

**Traveling Plan: "Adventure Odyssey"**

**Destination:** A 14-day journey to explore the breathtaking landscapes and vibrant cultures of New Zealand.

**Objective:** To create an unforgettable adventure that combines thrilling outdoor activities, cultural immersion, and relaxation in one of the world's most stunning countries.

**Day 1-2: Arrival in Auckland and Exploration**

* Arrive in Auckland, the City of Sails
* Visit the iconic Sky Tower for panoramic views of the city
* Explore the Auckland Art Gallery Toi o Tāmaki and the Auckland Museum
* Take a ferry to Waiheke Island for wine tasting and a scenic hike

**Day 3-4: Rotorua and Geothermal Wonders**

* Drive to Rotorua, a city steeped in Māori culture and geothermal activity
* Visit Wai-O-Tapu, a geothermal park with colorful hot springs and geysers
* Experience a traditional Māori hangi (feast) and cultural performance
* Explore the Redwood Forest and take a scenic hike

**Day 5-6: Tongariro National Park and Adventure**

* D

In [18]:
from subprocess import run

cmd='ollama list'
process=run(cmd,text=True,shell=True,capture_output=True)
print(process.stdout.strip())

NAME                       	ID          	SIZE  	MODIFIED    
llama3.1:latest            	91ab477bec9d	4.7 GB	13 days ago	
nomic-embed-text:latest    	0a109f422b47	274 MB	3 weeks ago	
llama3-groq-tool-use:latest	55065f5d86c6	4.7 GB	5 weeks ago	
mistral-nemo:latest        	4b300b8c6a97	7.1 GB	5 weeks ago	
llava-llama3:latest        	44c161b1f465	5.5 GB	6 weeks ago	
deepseek-coder-v2:latest   	8577f96d693e	8.9 GB	6 weeks ago
